In [1]:
%matplotlib inline

import catboost
import glob

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report, roc_auc_score


from word2vec import classification_tools
from protein_io import reader
from prot_features import prot_features
import matplotlib.pyplot as plt

sns.set(style="darkgrid")
pd.options.display.max_rows = 999

RANDOM_STATE = 42

model_path = '/home/pierre/riken/word2vec/prot_vec_model.model'
data_path = '/home/pierre/riken/data/riken_data/complete_from_xlsx.tsv'
BEST_MODEL_PROPERTIES_PATH = '/home/pierre/riken/data/riken_data/best_model.txt'
CV_OUTPUT_PATH = '/home/pierre/riken/data/riken_data/tree_depth_influence.csv'

K_FOLD = 10
agg_mode = 'sum'

# Import Data

In [12]:
df = pd.read_csv(data_path, sep='\t').dropna()
df = df.loc[df.seq_len >= 50, :]

X, y = df['sequences'].values, df['is_allergenic'].values
sentences = X.copy()
groups = df.species.values
# groups = LabelEncoder().fit_transform(df.species.values)

lenghts = df.seq_len.values

In [6]:
# def overlapping_tokenizer(seq, k=3):
#     """
#     'ABCDE' ==> ['ABC', 'BCE', 'CDE']
#     """
#     return [seq[idx:idx + k] for idx in range(len(seq) - k + 1)]
# X = [overlapping_tokenizer(seq) for seq in X]

In [13]:
featurizing = Pipeline([    
    ('ProteinTokenizer', classification_tools.ProteinTokenizer(token_size=3)),
    ('ProteinVectorization', classification_tools.ProteinW2VRepresentation(model_path=model_path, 
                                                                           agg_mode=agg_mode)),
])
X_w2v = featurizing.transform(np.array(X))

/home/pierre/miniconda3/envs/python3/lib/python3.6/site-packages/Bio/Seq.py:163: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


In [ ]:
gped = df.groupby(['genre'])
counts = gped.is_allergenic.count()

(counts.sample(frac=1).cumsum() / counts.sum()).head(20)

In [14]:
from sklearn.model_selection import GroupShuffleSplit, ShuffleSplit

train_inds, test_inds = next(GroupShuffleSplit(random_state=RANDOM_STATE).split(X_w2v, y, groups))
# train_inds, test_inds = next(ShuffleSplit(random_state=RANDOM_STATE).split(X_w2v, y, groups))
Xtrain, Xtest, ytrain, ytest = X_w2v[train_inds], X_w2v[test_inds], y[train_inds], y[test_inds]

train_1_inds, val_inds = next(GroupShuffleSplit(random_state=RANDOM_STATE).split(Xtrain, ytrain, groups[train_inds]))
# train_1_inds, val_inds = next(ShuffleSplit(random_state=RANDOM_STATE).split(Xtrain, ytrain, groups[train_inds]))
Xtrain_1, Xval, ytrain_1, yval = Xtrain[train_1_inds], Xtrain[val_inds], ytrain[train_1_inds], ytrain[val_inds]

print(Xtrain_1.shape, Xval.shape, Xtest.shape)

(10685, 100) (456, 100) (1325, 100)


In [15]:
train_inds

array([    0,     1,     3, ..., 12463, 12464, 12465])

# Dimension reduction & misc

In [6]:
# from sklearn.decomposition import IncrementalPCA

# pca = IncrementalPCA()
# pca.fit(X_w2v)
# X_pca = pca.transform(X_w2v)

# from sklearn.manifold import TSNE

# tsne = TSNE(verbose=3, perplexity=50).fit(X_pca)
# X_tsne = tsne.embedding_

# tsne_df = pd.DataFrame({'d1': X_tsne[:, 0], 'd2': X_tsne[:, 1], 'is_allergenic':y})

# sns.lmplot(x='d1', y='d2', data=tsne_df, fit_reg=False, hue='is_allergenic', legend=False)

# Gradient Tree Boosting

In [7]:
weights_uniform = [1.0, 1.0]
weights_balanced = [1.0, ((~y).sum()) / (y==True).sum()]

In [8]:
# clf = catboost.CatBoostClassifier(iterations=2000, verbose=False
# #                                   use_best_model=True
#                                  )

# grid = {
#     'depth': [10],  #np.arange(4, 11).tolist(),
# #     'C': np.geomspace(start=1e-3, stop=1e2, num=50)
# #     'l2_leaf_reg': [1.]
#     'l2_leaf_reg': [1.], #[1e-2, 1e-1, 1e0, 1e1, 1e2]
    
#     'class_weights': [weights_uniform]

# }

# cv = GridSearchCV(clf, 
#                   grid, 
#                   scoring=['accuracy', 'precision', 'recall', 'roc_auc'], 
#                   refit='roc_auc',
#                   cv=StratifiedKFold(n_splits=10, random_state=RANDOM_STATE),
#                   verbose=3, return_train_score=True)
# cv.fit(Xtrain, ytrain)

# # clf.fit(Xtrain, ytrain, eval_set=(Xtest, ytest))

In [7]:
clf = catboost.CatBoostClassifier(
    iterations=10000, od_type='Iter', use_best_model=True, od_wait=100,
    **{'bagging_temperature': 0.5, 'class_weights': [1.0, 5.0], 'depth': 5, 
     'l2_leaf_reg': 1.0, 'random_strength': 1}
    )
clf.fit(Xtrain_1, ytrain_1, eval_set=(Xval, yval))

0:	learn: 0.6792762	test: 0.6862006	best: 0.6862006 (0)	total: 68.8ms	remaining: 11m 28s
1:	learn: 0.6661237	test: 0.6794271	best: 0.6794271 (1)	total: 83.9ms	remaining: 6m 59s
2:	learn: 0.6542923	test: 0.6749414	best: 0.6749414 (2)	total: 99.5ms	remaining: 5m 31s
3:	learn: 0.6438843	test: 0.6697375	best: 0.6697375 (3)	total: 112ms	remaining: 4m 39s
4:	learn: 0.6318424	test: 0.6637213	best: 0.6637213 (4)	total: 124ms	remaining: 4m 7s
5:	learn: 0.6211150	test: 0.6570723	best: 0.6570723 (5)	total: 137ms	remaining: 3m 48s
6:	learn: 0.6118402	test: 0.6523397	best: 0.6523397 (6)	total: 150ms	remaining: 3m 34s
7:	learn: 0.6029598	test: 0.6453258	best: 0.6453258 (7)	total: 165ms	remaining: 3m 25s
8:	learn: 0.5950063	test: 0.6417825	best: 0.6417825 (8)	total: 177ms	remaining: 3m 16s
9:	learn: 0.5867560	test: 0.6360173	best: 0.6360173 (9)	total: 189ms	remaining: 3m 8s
10:	learn: 0.5769777	test: 0.6278671	best: 0.6278671 (10)	total: 206ms	remaining: 3m 7s
11:	learn: 0.5691820	test: 0.6243168	bes

In [8]:
from sklearn.metrics import classification_report, roc_auc_score

print(classification_report(ytest, clf.predict(Xtest)))
print('ROC AUC SCORE:', roc_auc_score(ytest, clf.predict_proba(Xtest)[:, 1]))

             precision    recall  f1-score   support

      False       0.83      0.81      0.82       902
       True       0.61      0.64      0.62       423

avg / total       0.76      0.76      0.76      1325

ROC AUC SCORE: 0.8128141823004305


             precision    recall  f1-score   support

      False       0.82      0.82      0.82       902
       True       0.61      0.62      0.62       423

avg / total       0.76      0.75      0.76      1325

ROC AUC SCORE: 0.7799714844343801

# More features

## Explainatory Power

In [16]:
Xtrain_mp = (Xtrain_mp - Xtrain_mp.mean(axis=0)) / Xtrain_mp.std(axis=0)
Xtest_mp = (Xtest_mp - Xtest_mp.mean(axis=0)) / Xtest_mp.std(axis=0)

In [19]:
logreg = LogisticRegressionCV(Cs=1000, class_weight='balanced')
logreg.fit(Xtrain_mp, ytrain)
print(classification_report(ytest, logreg.predict(Xtest_mp)))
print('ROC AUC SCORE:', roc_auc_score(ytest, logreg.predict_proba(Xtest_mp)[:, 1]))

             precision    recall  f1-score   support

      False       0.78      0.73      0.76       902
       True       0.50      0.57      0.53       423

avg / total       0.69      0.68      0.68      1325

ROC AUC SCORE: 0.6352995444848065


In [20]:
pd.DataFrame({'coef': logreg.coef_.reshape(-1), 'tag': feature_names})

,coef,tag
0,-0.534837,aliphatic_index
1,-0.268781,aromaticity
2,0.001740,gravy
3,-0.976391,instability_index
4,-0.652456,len_proteins
5,-0.415403,molecular_weight


## Boosting

In [8]:
new_features = [
    prot_features.aliphatic_index,
    prot_features.aromaticity,
    prot_features.gravy,
    prot_features.instability_index,
    prot_features.len_proteins,
    prot_features.molecular_weight
]

feature_names = [
    'aliphatic_index',
    'aromaticity',
    'gravy',
    'instability_index',
    'len_proteins',
    'molecular_weight',
]

In [9]:
X_molproperties = np.array([[feature(seq) for feature in new_features] for seq in sentences])

Xtrain_mp, Xtest_mp= X_molproperties[train_inds], X_molproperties[test_inds]
Xtrain_1_mp, Xval_mp = Xtrain_mp[train_1_inds], Xtrain_mp[val_inds]

Xtrain_all = np.concatenate([Xtrain, Xtrain_mp], axis=1)
Xtest_all = np.concatenate([Xtest, Xtest_mp], axis=1)
Xtrain1_all = np.concatenate([Xtrain_1, Xtrain_1_mp], axis=1)
Xval_all = np.concatenate([Xval, Xval_mp], axis=1)

In [15]:
import catboost

clf = catboost.CatBoostClassifier(
    iterations=10000, od_type='Iter', use_best_model=True, od_wait=100,
    **{'bagging_temperature': 0.5, 'class_weights': [1.0, 5.0], 'depth': 5, 
     'l2_leaf_reg': 1.0, 'random_strength': 1}
    )
clf.fit(Xtrain1_all, ytrain_1, eval_set=(Xval_all, yval))

0:	learn: 0.6796595	test: 0.6877583	best: 0.6877583 (0)	total: 13.3ms	remaining: 2m 13s
1:	learn: 0.6679301	test: 0.6796873	best: 0.6796873 (1)	total: 25.3ms	remaining: 2m 6s
2:	learn: 0.6567810	test: 0.6723880	best: 0.6723880 (2)	total: 37.7ms	remaining: 2m 5s
3:	learn: 0.6435894	test: 0.6641615	best: 0.6641615 (3)	total: 52.6ms	remaining: 2m 11s
4:	learn: 0.6281662	test: 0.6549625	best: 0.6549625 (4)	total: 66.7ms	remaining: 2m 13s
5:	learn: 0.6177161	test: 0.6497545	best: 0.6497545 (5)	total: 79.3ms	remaining: 2m 12s
6:	learn: 0.6052675	test: 0.6432567	best: 0.6432567 (6)	total: 93.3ms	remaining: 2m 13s
7:	learn: 0.5982680	test: 0.6401147	best: 0.6401147 (7)	total: 106ms	remaining: 2m 11s
8:	learn: 0.5897681	test: 0.6365095	best: 0.6365095 (8)	total: 120ms	remaining: 2m 12s
9:	learn: 0.5801472	test: 0.6313876	best: 0.6313876 (9)	total: 134ms	remaining: 2m 13s
10:	learn: 0.5699628	test: 0.6257784	best: 0.6257784 (10)	total: 148ms	remaining: 2m 14s
11:	learn: 0.5612442	test: 0.6211967

In [16]:
from sklearn.metrics import classification_report, roc_auc_score

print(classification_report(ytest, clf.predict(Xtest_all)))
print('ROC AUC SCORE:', roc_auc_score(ytest, clf.predict_proba(Xtest_all)[:, 1]))

             precision    recall  f1-score   support

      False       0.85      0.80      0.82       902
       True       0.61      0.69      0.65       423

avg / total       0.77      0.76      0.77      1325

ROC AUC SCORE: 0.820351936594801


In [ ]:
(pd.DataFrame({'coef': clf.feature_importances_, 'tag': [idx for idx in range(100)]+feature_names})
    .sort_values('coef', ascending=False)
    .head(10))